In [1]:
import assets.helper as b3
import assets.functions as run


In [2]:
import pandas as pd
import numpy as np
import os

import assets.graphs as g
import random
import requests

### QUOTES PRICES

In [ ]:
fund = run.sys_load_pkl(f'{b3.app_folder}/fund')
quotes = run.sys_load_pkl(f'{b3.app_folder}/quotes')


In [ ]:
fund.keys(), fund['BENS INDUSTRIAIS'].columns

In [ ]:
quotes.keys(), quotes['AERIS'].keys()

In [ ]:
df_preplot = {}
start_time = run.time.time()

for i, (setor, df_fund) in enumerate(fund.items()):
    df_fund = preprocess_data(df_fund)
    df_cd_conta = pivot_data(df_fund)

    df_unique = df_fund.drop_duplicates(subset=['DT_REFER', 'PREGAO']).drop(['CD_CONTA', 'DS_CONTA', 'VL_CONTA'], axis=1)
    df_premerged = pd.merge(df_unique, df_cd_conta, on=['DT_REFER', 'PREGAO'])
    df_resampled = resample_data(df_premerged)
    
    df_merged = merge_with_bigdata(df_resampled, bigdata)
    df_merged = df_merged.set_index('Date', drop=True)
    df_merged = df_merged.groupby('PREGAO', group_keys=False).apply(add_metrics)

    # Clean up the dataframe using the cleanup_dataframe function
    df_merged = cleanup_dataframe(df_merged)

    df_merged = add_metrics(df_merged)

    df_preplot[setor] = df_merged

    print(setor, run.sys_remaining_time(start_time, len(fund), i))


In [ ]:
# Define the path to the folder
company_folder = 'company'
folder_path = os.path.join(b3.app_folder, company_folder)

# Check if the folder does not exist
if not os.path.exists(folder_path):
    # Create the folder
    os.makedirs(folder_path)


In [ ]:
start_time = run.time.time()
for i, (setor, df) in enumerate(df_preplot.items()):
    companies = df['PREGAO'].unique()
    for i2, company in enumerate(companies):
        mask = df['PREGAO'] == company
        df_temp = df[mask]
        try:
            df_temp = run.sys_save_pkl(df_temp, f'{b3.app_folder}/{company_folder}/{company}')
        except Exception as e:
            pass
    print(setor, run.sys_remaining_time(start_time, len(df_preplot), i))


In [ ]:
mask = df_preplot['BENS INDUSTRIAIS']['TICKER'] == 'AERI3'
df_preplot['BENS INDUSTRIAIS'][mask]

### DASH

In [ ]:
company = 'AZEVEDO'
df = run.sys_load_pkl(f'{b3.app_folder}/{b3.company_folder}/{company}')
tickers = df['TICKER'].unique()
# df.columns.to_list()

In [ ]:
df['03.11 - Lucro Líquido']
df['14.04.01 - ROE (Resultado por Patrimônio)']
df['03.01 - Receita Bruta']
df['06.01 - Caixa das Operações']





In [ ]:
graphs = g.construct_graphs(df)
graphs.keys()

In [ ]:
import assets.lines
lines = assets.lines.report
groups = set(entry['title'][:2] for entry in lines)
groups

In [ ]:
for g, (group_key, group) in enumerate(graphs.items()):
    status = True if g == 0 else False
    plots = []
    for l, (line_key, line) in enumerate(group.items()):
        for p, (plot_key, plot_info) in enumerate(line.items()):
            for ticker in df['TICKER'].unique():
                df_ticker = df[df['TICKER'] == ticker]
                if not df.empty:
                    print(group[0][0]['info']['title'])
                    print(line[0]['info']['title'])
                    print(plot_info['info']['title'])
                    run.plot_tweak(plot_info, df_ticker).show();

In [ ]:
for ticker in tickers:
    df = df[df['TICKER'] == ticker]
    print(ticker, len(df))

    if not df.empty:
        for g, (group_key, group) in enumerate(merged_entries.items()):
            status = True if g == 0 else False
            plots = []
            for l, (line_key, line) in enumerate(group.items()):
                for p, (plot_key, plot_info) in enumerate(line.items()):
                    print(ticker)
                    run.plot_tweak(plot_info, df).show();

In [ ]:
fund = run.load_database()


In [ ]:
setores = []
for setor, df in fund.items():
    columns=['SETOR', 'SUBSETOR', 'SEGMENTO', 'PREGAO']
    setores.append(df[columns].drop_duplicates())
    print(setor)
sss = pd.concat(setores)
sss = sss.applymap(run.clean_text)

sss = run.sys_save_pkl(sss, f'{b3.app_folder}/sss')

In [ ]:
sss

In [ ]:
mask = sss['SUBSETOR'] == 'MATERIAL DE TRANSPORTE'
sss[mask]

#### Playground


In [ ]:
company_list = run.sys_load_pkl('company_list')
company_list

In [ ]:
listagem_siglas = ['NM', 'N1', 'N2', 'MA', 'M2', 'MB', 'DR1', 'DR2', 'DR3', 'DRE', 'DRN']
listagem_extenso = ['Novo Mercado', 'Nível 1 de Governança Corporativa', 'Nível 2 de Governança Corporativa', 'Bovespa Mais', 'Bovespa Mais Nível 2', 'Organização Tradicional OTC', 'BDR Nível 1', 'BDR Nível 2', 'BDR Nível 3', 'BDR ETF', 'BDR Não Patrocinado']
list_dict = dict(zip(listagem_siglas, listagem_extenso))
list_dict

#### Merge and update


In [ ]:
driver, wait = run.sys_load_browser()
url = 'https://sistemaswebb3-listados.b3.com.br/listedCompaniesPage/search?language=pt-br' 



In [ ]:
b3_cols = b3.cols_b3_companies + b3.col_b3_companies_extra_columns
companies__local = run.sys_read_or_create_dataframe('company', b3_cols).fillna('')
b3_companies_tickers = run.sys_load_pkl('b3_companies_tickers')

b3_companies_tickers = run.get_b3_tickers(driver, wait, url)
for col in b3_cols:
    if col not in b3_companies_tickers.columns:
        b3_companies_tickers[col] = ''
b3_companies_tickers = b3_companies_tickers[b3_cols]
# b3_companies_tickers = sys_save_pkl(b3_companies_tickers, 'b3_companies_tickers')

key_columns = ['ticker', 'company_name']
len(b3_companies_tickers), len(companies__local)

In [ ]:
merged = pd.merge(companies__local, b3_companies_tickers[:], how='outer', on=key_columns, indicator=True)

update_strict = merged[merged['_merge'] == 'right_only'][key_columns] # only companies new in web
update_broad = merged[merged['_merge'] != 'left_only'][key_columns] # all companies from web
updated = update_strict

len(update_broad), len(update_strict)

In [ ]:
key_columns = ['company_name', 'ticker']
b3_cols = b3.cols_b3_companies + b3.col_b3_companies_extra_columns
b3_cols_float = ['Capital Social']
b3_str_cols = [col for col in b3_cols if col not in  b3_cols_float]
col_types = {col: 'float' if col in b3_cols_float else 'str' for col in b3_cols}


In [ ]:
driver.get(b3.url)
# time.sleep(1)

size = len(updated)
# Iterate through the new companies to extract detailed information
new_companies = []
start_time = run.time.time()
for i, (index, row) in enumerate(updated.iterrows()):
    driver.get(b3.url)
    new_company = run.b3_get_company_info(row, driver, wait)
    new_company = pd.DataFrame([new_company], columns=b3_cols).astype(col_types)
    new_company['Capital Social'] = new_company['Capital Social'].replace(np.nan, 0.0)
    new_company = new_company.replace('nan', '')
    new_company['Capital Social'] = pd.to_numeric(new_company['Capital Social'], errors='coerce').astype('float')

    try:
        if not new_company.empty:
            cnpj = new_company['cnpj'][0]
            url = f'https://cnpj.biz/{cnpj}'
            response = requests.get(url, headers={"User-Agent": random.choice(b3.USER_AGENTS)})
            extra = run.b3_get_company_extra_cnpj_info(response)
            extra = pd.DataFrame([extra], columns=b3_cols).fillna('')
            extra = extra.astype(col_types)
        else:
            extra = pd.DataFrame([], columns=b3_cols)

    except Exception as e:
        pass
    extra = extra.reset_index(drop=True)
    new_company = pd.merge(new_company[b3.cols_b3_companies], extra[b3.col_b3_companies_extra_columns], left_on='cnpj', right_on='CNPJ', how='outer').fillna('').reset_index(drop=True)
    new_companies.append(new_company)
    
    print(run.sys_remaining_time(start_time, size, i), row['ticker'], row['company_name'])

    if (size-i-1) % (b3.bin_size/10) == 0:
        temp = pd.concat(new_companies).reset_index(drop=True)
        temp['Capital Social'] = pd.to_numeric(temp['Capital Social'], errors='coerce').astype('float')
        temp['Capital Social'] = temp['Capital Social'].replace(np.nan, 0.0)
        temp = pd.merge(companies__local, temp, on=b3_cols, how='outer', indicator=False).fillna('').drop_duplicates(subset=key_columns, keep='last').reset_index(drop=True)
        temp = sys_save_and_pickle(temp, 'company')


In [ ]:
temp

In [ ]:
for col in b3_cols:
    d1 = new_company[col].dtype
    d2 = extra[col].dtype
    print(col, d1, d2)
    if d1 != d2:
        print(col, d1, d2)
    

In [ ]:
b3_cols

##### CVM GET CODE

In [ ]:
for i in range(0,1000+1):
    url = f'https://sistemaswebb3-listados.b3.com.br/listedCompaniesPage/main/{i}/ABCD/overview?language=pt-br'
    print(url)

##### Final Company Get

In [ ]:
companies__local = run.sys_read_or_create_dataframe('company', b3_cols)


In [ ]:
companies__local.columns

In [ ]:
mask = companies__local['Estado'] == ''
# mask &= companies__local['cnpj'] != ''
companies__local = companies__local[~mask]

In [ ]:
companies__local = run.sys_save_and_pickle(companies__local, 'companies__local')

### Playground

In [ ]:
filename = 'acoes'
columns = ['Companhia', 'Trimestre', 'Ações ON', 'Ações PN', 'Ações ON em Tesouraria', 'Ações PN em Tesouraria', 'Unidade', 'URL']
acoes = run.sys_read_or_create_dataframe(filename, columns)
acoes.to_csv('acoes.csv')


In [ ]:
filename = 'nsd_links'
cols_nsd = ['company', 'dri', 'dri2', 'dre', 'data', 'versao', 'auditor', 'auditor_rt', 'cancelamento', 'protocolo', 'envio', 'url', 'nsd']

nsd = run.sys_read_or_create_dataframe(filename, cols_nsd)
nsd.to_csv('nsd_links.csv')

In [85]:
filename = 'company'
columns = ['pregao', 'company_name', 'cvm', 'listagem', 'ticker', 'tickers',
       'asin', 'cnpj', 'site', 'setor', 'subsetor', 'segmento', 'atividade',
       'escriturador', 'stock_holders', 'url', 'CNPJ', 'Razão Social',
       'Nome Fantasia', 'Data da Abertura', 'Porte', 'Natureza Jurídica',
       'Opção pelo MEI', 'Opção pelo Simples', 'Capital Social', 'Tipo',
       'Situação', 'Logradouro', 'Bairro', 'CEP', 'Município', 'Estado']

company = run.sys_read_or_create_dataframe(filename, columns)
company.to_csv('company.csv')

company: total 301 items


In [86]:
mask = company['Situação'] == 'Ativa'
mask &= company['setor'] != 'NENHUM'
mask &= company['setor'] != 'NAO CLASSIFICADOS'
mask &= company['setor'] != 'OUTROS'

company[mask]


,pregao,company_name,cvm,listagem,ticker,tickers,asin,cnpj,site,setor,...,Opção pelo MEI,Opção pelo Simples,Capital Social,Tipo,Situação,Logradouro,Bairro,CEP,Município,Estado
1,3R PETROLEUM,3R PETROLEUM OLEO E GAS SA,,CIA NOVO MERCADO,RRRP,RRRP3,,12.091.809/0001-55,,PETROLEO GAS E BIOCOMBUSTIVEIS,...,Não,Não,5.054406e+09,Matriz,Ativa,"Praia Botafogo, 186",Botafogo,22250-145,Rio de Janeiro,Rio de Janeiro
27,AERIS,AERIS IND E COM DE EQUIP GERACAO DE ENERGIA SA,,CIA NOVO MERCADO,AERI,AERI3,,12.528.708/0001-07,,BENS INDUSTRIAIS,...,Não,Não,8.719750e+08,Matriz,Ativa,"Rodovia Ce 155, S/N",Complexo Industrial e Portuario do Pecem,61680-000,Caucaia,Ceará
30,AES BRASIL,AES BRASIL ENERGIA SA,,CIA NOVO MERCADO,AESB,AESB3,,37.663.076/0001-07,http://ri.aesbrasil.com.br,UTILIDADE PUBLICA,...,Não,Não,2.196958e+09,Matriz,Ativa,"Avenida Eng Luiz Carlos Berrini, 1376",Cidade Moncoes,04571-936,São Paulo,São Paulo
31,AESOPERACOES,AES BRASIL OPERACOES SA,,,AESO,AESO3,,00.194.724/0001-13,http://ri.aesbrasil.com.br,UTILIDADE PUBLICA,...,Não,Não,1.799262e+09,Matriz,Ativa,"Rodovia Comandante Joao Ribeiro de Barros, S/N",Distrito Industrial Marcus Vinicius Feliz Machado,17039-800,Bauru,São Paulo
36,AFLUENTE T,AFLUENTE TRANSMISSAO DE ENERGIA ELETRICA SA,,,AFLT,AFLT3,,10.338.320/0001-00,http://www.afluentet.com.br/,UTILIDADE PUBLICA,...,Não,Não,6.308470e+07,Matriz,Ativa,"Praia Do Flamengo, 78",Flamengo,22210-030,Rio de Janeiro,Rio de Janeiro
51,AGROGALAXY,AGROGALAXY PARTICIPACOES SA,,CIA NOVO MERCADO,AGXY,AGXY3,,21.240.146/0001-84,,CONSUMO NAO CICLICO,...,Não,Não,8.387171e+08,Matriz,Ativa,"Rua Iguatemi, 192",Itaim Bibi,01451-010,São Paulo,São Paulo
80,ALFA HOLDING,ALFA HOLDINGS SA,,,RPAD,"RPAD3, RPAD5, RPAD6",,17.167.396/0001-69,www.alfanet.com.br,FINANCEIRO,...,Não,Não,4.302450e+08,Matriz,Ativa,"Alameda Santos, 466",Paraiso,01418-000,São Paulo,São Paulo
82,ALGAR TELEC,ALGAR TELECOM SA,,,ALGT,ALGT-DEB62,,71.208.516/0001-74,http://www.algartelecom.com.br,COMUNICACOES,...,Não,Não,9.018312e+08,Matriz,Ativa,"Rua Jose Alves Garcia, 415",Brasil,38400-668,Uberlândia,Minas Gerais
87,ALLIAR,ALLIANCA SAUDE E PARTICIPACOES SA,,CIA NOVO MERCADO,AALR,AALR3,,42.771.949/0001-35,http://ri.alliar.com/,SAUDE,...,Não,Não,6.353720e+08,Filial,Ativa,"Avenida Bernardo Monteiro, 1472",Funcionarios,30150-288,Belo Horizonte,Minas Gerais
88,ALLIED,ALLIED TECNOLOGIA SA,,CIA NOVO MERCADO,ALLD,ALLD3,,20.247.322/0001-47,,CONSUMO CICLICO,...,Não,Não,1.021575e+09,Filial,Ativa,"Rodovia Vice-Prefeito Hermenegildo Tonolli, 1.500",Distrito Industrial,13213-086,Jundiaí,São Paulo
